In [1]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import requests

In [2]:
from_github = True
url = 'https://raw.githubusercontent.com/hodcroftlab/covariants/master/web/data/perClusterData.json'

In [3]:
if from_github:
    countries = requests.get(url).json()['country_names']

else:
    with open('perClusterData.json', 'r') as f:
        countries = json.load(f)['country_names']

In [4]:
if from_github:
    data = requests.get(url).json()['distributions']
    results_df = pd.DataFrame.from_dict(data)

else:
    with open('perClusterData.json', 'r') as f:
        data = json.load(f)['distributions']
        results_df = pd.DataFrame.from_dict(data)

In [5]:
cols = ['frequencies', 'interp', 'orig', 'week']

### Choose variants

In [6]:
list_VOC = ['21A (Delta)','21I (Delta)','21J (Delta)','21K (Omicron)','21L (Omicron)']

voc_idx = []
for voc in list_VOC:
    voc_idx.append(pd.Series.to_list(results_df['cluster']).index(voc))

In [7]:
variants = {}


for country in countries:
    
    cnt = 0

    df_country = pd.DataFrame(columns = ['week','country'])
    
    for j in voc_idx:
        df_variant = pd.DataFrame(columns = cols)


        for i in range(len(results_df.distribution[j])):

            df = pd.DataFrame.from_dict(dict(results_df.distribution[j][i]))


            if country in df.index:
                df_variant = df_variant.append(df.loc[df.index == country])

                
        variant = results_df['cluster'][j]
        #print(variant)
        #print(df_variant)
        
        if cnt == 0:

            df_country[variant] = df_variant['frequencies']
            df_country['country'] = country
            df_country['week'] = df_variant['week']
            
            cnt +=1
        else:
            df_country[variant] = df_variant['frequencies']

            #print(df_country)
        
    variants[country] = df_country
    df_temp = pd.DataFrame()
    df_temp = df_country
    df_temp = df_temp.set_index('week')
    df_temp = df_temp.drop(columns = ['country'])
    df_temp.to_csv('Variants\\' + country + '.csv')

